# SDCFE Model
- Seperate Dense Concat Fold Ensenble Model
- 연관이 있는 열을 하나로 묶고, 연관성이 있는 그룹들 각각을 Dense layer로 차원을 축소하여 잠재벡터를 만듦.
- 이후, 해당 잠재 벡터들을 모두 병합함(Concat)
- KFOLD 5 로 5개의 모델을 각각 생성하고, 모델들을 저장한 후, Ensemble Method를 사용하여 5개의 모델을 모두 앙상블 시킴

### 패키지 로딩

In [14]:
# Data Handling
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt

# To ignore warnings
import warnings 
warnings.filterwarnings('ignore')

# fix seed
import random
import os
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42)

# Tensorflow and Keras
import tensorflow as tf
from tensorflow import keras
import keras
from keras.utils import np_utils
from keras.models import Sequential,Model,load_model
from keras.layers import Dense,Dropout,BatchNormalization,Add,Input,Activation,Concatenate,Average
from keras.layers.merge import concatenate

# scoring
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import sklearn.metrics as metrics

### 데이터 로딩

In [15]:
X = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/X_train.csv")
y = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/y_train.csv")

### Seperate
- 연관이 있는 열을 하나로 묶음

In [16]:
# 연관이 있는 열을 하나로 묶는 과정
PCB_press = X[["X_01","X_02","X_05","X_06"]]
material_weight = X[["X_03","X_10","X_11"]]
material_area = X[["X_07","X_08","X_09"]]
pad_loc = X[["X_14","X_15","X_16","X_17","X_18","X_13"]]
screw1_deep = X[["X_19","X_20","X_21","X_22"]]
connect_pin = X[["X_24","X_25","X_26","X_27","X_28","X_29","X_12"]]
screw2_deep = X[["X_30","X_31","X_32","X_33"]]
screw_rotate = X[["X_34","X_35","X_36","X_37"]]
housing = X[["X_38","X_39","X_40","X_46"]]
radom = X[["X_41","X_42","X_43","X_44","X_45"]]
RF = X[["X_49","X_50","X_51","X_52","X_53","X_54","X_55","X_56"]]

# 딥러닝 모델 학습을 위해 넘파이 형태로 데이터 변환
PCB_press = np.array(PCB_press)
material_weight = np.array(material_weight)
material_area = np.array(material_area)
pad_loc = np.array(pad_loc)
screw1_deep = np.array(screw1_deep)
connect_pin = np.array(connect_pin)
screw2_deep = np.array(screw2_deep)
screw_rotate = np.array(screw_rotate)
housing = np.array(housing)
radom = np.array(radom)
RF = np.array(RF)
y = np.array(y)

### 모델 구축

In [17]:
### input 정의 : 연관 있는 열들을 모두 각각의 input으로 형성
PCB_press_input = keras.layers.Input(shape=(4,), name="PCB_press_input", dtype="float32")
material_weight_input = keras.layers.Input(shape=(3,), name="material_weight_input", dtype="float32")
material_area_input = keras.layers.Input(shape=(3,), name="material_area_input", dtype="float32")
pad_loc_input = keras.layers.Input(shape=(6,), name="pad_loc_input", dtype="float32")
screw1_deep_input = keras.layers.Input(shape=(4,), name="screw1_deep_input", dtype="float32")
connect_pin_input = keras.layers.Input(shape=(7,), name="connect_pin_input", dtype="float32")
screw2_deep_input = keras.layers.Input(shape=(4,), name="screw2_deep_input", dtype="float32")
screw_rotate_input = keras.layers.Input(shape=(4,), name="screw_rotate_input", dtype="float32")
housing_input = keras.layers.Input(shape=(4,), name="housing_input", dtype="float32")
radom_input = keras.layers.Input(shape=(5,), name="radom_input", dtype="float32")
RF_input = keras.layers.Input(shape=(8,), name="RF_input", dtype="float32")

In [18]:
### Dense layer를 이용하여 Seperate 된 각각의 열들의 집합들의 잠재벡터를 하나씩 추출해줌

# PCB_press
PCB_press_layer1 = keras.layers.Dense(256, activation="relu")(PCB_press_input)
PCB_press_layer2 = keras.layers.Dense(128, activation="relu")(PCB_press_layer1)
PCB_press_layer3 = keras.layers.Dense(256, activation="relu")(PCB_press_layer2)
PCB_press_layer4 = keras.layers.Dense(4, activation="relu")(PCB_press_layer3)

# material_weight
material_weight_layer1 = keras.layers.Dense(256, activation="relu")(material_weight_input)
material_weight_layer2 = keras.layers.Dense(128, activation="relu")(material_weight_layer1)
material_weight_layer3 = keras.layers.Dense(256, activation="relu")(material_weight_layer2)
material_weight_layer4 = keras.layers.Dense(3, activation="relu")(material_weight_layer3)

# material_area
material_area_layer1 = keras.layers.Dense(256, activation="relu")(material_area_input)
material_area_layer2 = keras.layers.Dense(128, activation="relu")(material_area_layer1)
material_area_layer3 = keras.layers.Dense(256, activation="relu")(material_area_layer2)
material_area_layer4 = keras.layers.Dense(3, activation="relu")(material_area_layer3)

# pad_loc
pad_loc_layer1 = keras.layers.Dense(256, activation="relu")(pad_loc_input)
pad_loc_layer2 = keras.layers.Dense(128, activation="relu")(pad_loc_layer1)
pad_loc_layer3 = keras.layers.Dense(256, activation="relu")(pad_loc_layer2)
pad_loc_layer4 = keras.layers.Dense(6, activation="relu")(pad_loc_layer3)

# screw1_deep
screw1_deep_layer1 = keras.layers.Dense(256, activation="relu")(screw1_deep_input)
screw1_deep_layer2 = keras.layers.Dense(128, activation="relu")(screw1_deep_layer1)
screw1_deep_layer3 = keras.layers.Dense(256, activation="relu")(screw1_deep_layer2) 
screw1_deep_layer4 = keras.layers.Dense(4, activation="relu")(screw1_deep_layer3)

# connect_pin
connect_pin_layer1 = keras.layers.Dense(256, activation="relu")(connect_pin_input)
connect_pin_layer2 = keras.layers.Dense(128, activation="relu")(connect_pin_layer1)
connect_pin_layer3 = keras.layers.Dense(256, activation="relu")(connect_pin_layer2)
connect_pin_layer4 = keras.layers.Dense(7, activation="relu")(connect_pin_layer3)

# screw2_deep
screw2_deep_layer1 = keras.layers.Dense(256, activation="relu")(screw2_deep_input)
screw2_deep_layer2 = keras.layers.Dense(128, activation="relu")(screw2_deep_layer1)
screw2_deep_layer3 = keras.layers.Dense(256, activation="relu")(screw2_deep_layer2)
screw2_deep_layer4 = keras.layers.Dense(4, activation="relu")(screw2_deep_layer3)

# screw_rotate
screw_rotate_layer1 = keras.layers.Dense(256, activation="relu")(screw_rotate_input)
screw_rotate_layer2 = keras.layers.Dense(128, activation="relu")(screw_rotate_layer1)
screw_rotate_layer3 = keras.layers.Dense(256, activation="relu")(screw_rotate_layer2)
screw_rotate_layer4 = keras.layers.Dense(4, activation="relu")(screw_rotate_layer3)

# housing
housing_layer1 = keras.layers.Dense(256, activation="relu")(housing_input)
housing_layer2 = keras.layers.Dense(128, activation="relu")(housing_layer1)
housing_layer3 = keras.layers.Dense(256, activation="relu")(housing_layer2)
housing_layer4 = keras.layers.Dense(4, activation="relu")(housing_layer3)

# radom
radom_layer1 = keras.layers.Dense(256, activation="relu")(radom_input)
radom_layer2 = keras.layers.Dense(128, activation="relu")(radom_layer1)
radom_layer3 = keras.layers.Dense(256, activation="relu")(radom_layer2)
radom_layer4 = keras.layers.Dense(5, activation="relu")(radom_layer3)

# RF
RF_layer1 = keras.layers.Dense(256, activation="relu")(RF_input)
RF_layer2 = keras.layers.Dense(128, activation="relu")(RF_layer1)
RF_layer3 = keras.layers.Dense(256, activation="relu")(RF_layer2)
RF_layer4 = keras.layers.Dense(8, activation="relu")(RF_layer3)

### 생성된 잠재벡터들을 하나의 layer에서 모두 병합시켜줌
# concat
concat_layer = keras.layers.concatenate([PCB_press_layer4, material_weight_layer4, material_area_layer4, pad_loc_layer4, screw1_deep_layer4, connect_pin_layer4, screw2_deep_layer4, screw_rotate_layer4, housing_layer4, radom_layer4, RF_layer4])

### 결과가 나오도록 함
# result
result_layer = keras.layers.Dense(14)(concat_layer)

In [21]:
### 모델 적합 함수 생성
def fit_model(PCB_press_train,material_weight_train,material_area_train,pad_loc_train,screw1_deep_train,connect_pin_train,screw2_deep_train,screw_rotate_train,housing_train,radom_train,RF_train,PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val,y_train,y_val,i): 
    ### 모델 정의
    model = keras.Model(
        inputs=[PCB_press_input,material_weight_input,material_area_input,pad_loc_input,screw1_deep_input,connect_pin_input,screw2_deep_input,screw_rotate_input,housing_input,radom_input,RF_input], 
        outputs=result_layer)
    
    ### 모델 컴파일
    model.compile(loss='mse', optimizer='adam')

    ### early stopping option 걸기
    early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20)
    
    ### checkpoint 옵션 걸기 -> fold 내에서 모델이 개선되는 시점의 모델을 h5 형태로 저장시킴
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath = 'C:/LG_Aimers/Conv/model/'+'fold' + str(i) + '--.{val_loss:.3f}'+'.h5', 
        monitor='val_loss', 
        save_best_only=True)

    ### model fitting
    model.fit([PCB_press_train,material_weight_train,material_area_train,pad_loc_train,screw1_deep_train,connect_pin_train,screw2_deep_train,screw_rotate_train,housing_train,radom_train,RF_train], 
              y_train,
              batch_size=32,
              epochs=100, 
              validation_data=([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val], y_val), 
              callbacks=[early_stopping, checkpoint])
    return model

In [22]:
### KFOLD 5 함수 생성 및 적용

kf=KFold(5,shuffle=True)
i=0
for train_index,val_index in kf.split(X):
  i+=1
  PCB_press_train,PCB_press_val=PCB_press[train_index],PCB_press[val_index]
  material_weight_train,material_weight_val=material_weight[train_index],material_weight[val_index]
  material_area_train,material_area_val=material_area[train_index],material_area[val_index]
  pad_loc_train,pad_loc_val=pad_loc[train_index],pad_loc[val_index]
  screw1_deep_train,screw1_deep_val=screw1_deep[train_index],screw1_deep[val_index]
  connect_pin_train,connect_pin_val=connect_pin[train_index],connect_pin[val_index]
  screw2_deep_train,screw2_deep_val=screw2_deep[train_index],screw2_deep[val_index]
  screw_rotate_train,screw_rotate_val=screw_rotate[train_index],screw_rotate[val_index]
  housing_train,housing_val=housing[train_index],housing[val_index]
  radom_train,radom_val=radom[train_index],radom[val_index]
  RF_train,RF_val=RF[train_index],RF[val_index]
  
  y_train,y_val=y[train_index],y[val_index]
  model=fit_model(PCB_press_train,material_weight_train,material_area_train,pad_loc_train,screw1_deep_train,connect_pin_train,screw2_deep_train,screw_rotate_train,housing_train,radom_train,RF_train,PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val,y_train,y_val,i)
  print('finish {} model'.format(i))

Epoch 1/100
793/793 [==============================] - 9s 10ms/step - loss: 274.6530 - val_loss: 1.7292
Epoch 2/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6684 - val_loss: 1.7160
Epoch 3/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6365 - val_loss: 1.6474
Epoch 4/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6231 - val_loss: 1.6986
Epoch 5/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6231 - val_loss: 1.6208
Epoch 6/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6107 - val_loss: 1.6214
Epoch 7/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6025 - val_loss: 1.5963
Epoch 8/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6023 - val_loss: 1.5881
Epoch 9/100
793/793 [==============================] - 7s 9ms/step - loss: 1.6016 - val_loss: 1.8346
Epoch 10/100
793/793 [==============================] - 7s 9ms/step - loss: 1.5986 - val

### Ensemble

In [25]:
### 5개의 앙상블 모델 로딩
model1=load_model("C:/LG_Aimers/Conv/model/fold1--.1.557.h5")
print(model1.evaluate([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val))

model2=load_model("C:/LG_Aimers/Conv/model/fold2--.1.619.h5")
print(model2.evaluate([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val))

model3=load_model("C:/LG_Aimers/Conv/model/fold3--.1.568.h5")
print(model3.evaluate([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val))

model4=load_model("C:/LG_Aimers/Conv/model/fold4--.1.466.h5")
print(model4.evaluate([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val))

model5=load_model("C:/LG_Aimers/Conv/model/fold5--.1.508.h5")
print(model5.evaluate([PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val))

199/199 [==============================] - 1s 5ms/step - loss: 1.5020
1.5020248889923096
199/199 [==============================] - 1s 4ms/step - loss: 1.5019
1.5019190311431885
199/199 [==============================] - 1s 4ms/step - loss: 1.5012
1.5012356042861938
199/199 [==============================] - 1s 3ms/step - loss: 1.5004
1.5004385709762573
199/199 [==============================] - 1s 5ms/step - loss: 1.5080
1.5080111026763916


In [29]:
### 5개의 앙상블 모델 합치기
all_models=list([model1,model2,model3,model4,model5])
  
def define_stacked_model(members):
  # update all layers in all models to not be trainiable
  for i in range(len(members)):
    model=members[i]
    for layer in model.layers:
      layer.trainable=False
      # rename to avoid 'unique layer name' issue
      layer._name = 'ensemble_' + str(i+1) + '_' + layer.name
  
  # define multi-handed input
  ensemble_visible=[model.input for model in members]
  ensemble_outputs=[model.output for model in members]

  y=Average()(ensemble_outputs)

  model=Model(inputs=ensemble_visible,outputs=y,name='ensemble')

  keras.utils.plot_model(model,show_shapes=True,to_file='C:\LG_Aimers\XGB노가다\Y_03_결과\model_graph.jpg')

  model.compile(loss='mae',optimizer='adam')
  return model

def fit_stacked_model(model,trainX,valX,trainY,valY):
  x_train=[trainX for _ in range(len(model.input))]
  x_val=[valX for _ in range(len(model.input))]
  y_train=trainY
  y_val=valY

  es=keras.callbacks.EarlyStopping(monitor='val_loss',patience=20)
  checkpoint_filepath='/content/drive/MyDrive/lg aimers/ensemble_model.h5'
  cp=keras.callbacks.ModelCheckpoint(
      filepath=checkpoint_filepath,
      monitor='val_loss',
      save_best_only=True
  )
  model.fit(x_train,y_train,epochs=1,validation_data=(x_val,y_val),callbacks=[es,cp],batch_size=32)

def evaluate_stacked_model(model,inputX,y_val):
  x_val=[inputX for _ in range(len(model.input))]
  return model.evaluate(x_val,y_val)

def predict_stacked_model(model,inputX):
  x_test=[inputX for _ in range(len(model.input))]
  return model.predict(x_test)

In [30]:
### 모델 스태킹하기
n_members=5
members=all_models

stacked_model=define_stacked_model(members)
fit_stacked_model(stacked_model,[PCB_press_train,material_weight_train,material_area_train,pad_loc_train,screw1_deep_train,connect_pin_train,screw2_deep_train,screw_rotate_train,housing_train,radom_train,RF_train],[PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_train,y_val)

793/793 [==============================] - 18s 19ms/step - loss: 0.7130 - val_loss: 0.7032


In [33]:
### Stacking 모델 평가하기
evaluate_stacked_model(stacked_model,[PCB_press_val,material_weight_val,material_area_val,pad_loc_val,screw1_deep_val,connect_pin_val,screw2_deep_val,screw_rotate_val,housing_val,radom_val,RF_val],y_val)

199/199 [==============================] - 3s 15ms/step - loss: 0.7032


0.7031537890434265

### 모델 평가

In [52]:
### 따로 나눈 데이터로 테스트 ###
X_valid = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/X_train.csv")
y_valid = pd.read_csv("C:/LG_Aimers/data/Train-Test-Split/y_train.csv")

In [37]:
### Seperate
PCB_press_valid = X_valid[["X_01","X_02","X_05","X_06"]]
material_weight_valid = X_valid[["X_03","X_10","X_11"]]
material_area_valid = X_valid[["X_07","X_08","X_09"]]
pad_loc_valid = X_valid[["X_14","X_15","X_16","X_17","X_18","X_13"]]
screw1_deep_valid = X_valid[["X_19","X_20","X_21","X_22"]]
connect_pin_valid = X_valid[["X_24","X_25","X_26","X_27","X_28","X_29","X_12"]]
screw2_deep_valid = X_valid[["X_30","X_31","X_32","X_33"]]
screw_rotate_valid = X_valid[["X_34","X_35","X_36","X_37"]]
housing_valid = X_valid[["X_38","X_39","X_40","X_46"]]
radom_valid = X_valid[["X_41","X_42","X_43","X_44","X_45"]]
RF_valid = X_valid[["X_49","X_50","X_51","X_52","X_53","X_54","X_55","X_56"]]

In [38]:
### Stacking
result=predict_stacked_model(stacked_model,[PCB_press_valid,material_weight_valid,material_area_valid,pad_loc_valid,screw1_deep_valid,connect_pin_valid,screw2_deep_valid,screw_rotate_valid,housing_valid,radom_valid,RF_valid])

In [50]:
### 정답 데이터
y_valid_array = np.array(y_valid)

In [46]:
### Scoring 함수 생성
def lg_nrmse(gt, preds):
    # 각 Y Feature별 NRMSE 총합
    # Y_01 ~ Y_08 까지 20% 가중치 부여
    all_nrmse = []
    for idx in range(0,14): # ignore 'ID'
        rmse = metrics.mean_squared_error(gt[:,idx], preds[:,idx], squared=False)
        nrmse = rmse/np.mean(np.abs(gt[:,idx]))
        all_nrmse.append(nrmse)
    score = 1.2 * np.sum(all_nrmse[:8]) + 1.0 * np.sum(all_nrmse[8:15])
    return score

In [51]:
### 결과 도출
lg_nrmse(y_valid_array, result)

2.0156351596250097